In [19]:
import pandas as pd
import altair as alt

In [20]:
dados_federais_campina = pd.read_csv("data\\dados_federais_obitos_cg.csv", encoding='utf-8', error_bad_lines=False)
dados_painelfederal_campina = pd.read_csv("data\\dados_painel-federal_obitos_cg.csv", encoding='UTF-8', error_bad_lines=False)
dados_municipais_campina = pd.read_csv("data\\dados_municipais_cg.csv", encoding='utf-8', index_col='data')

C:\Users\perei\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


# Processando os dados sobre o Numero de Obitos por Dia (Painel Federal)

In [21]:
# Criando um data com a coluna 'data' e a coluna 'obitosNovos'
pf_obitosPorDia = dados_painelfederal_campina[['obitosNovos', 'data']]


In [22]:
# Criando um DataFrame com os dados para o altair
pf_obitosPorDia = pd.DataFrame({
    'data': pf_obitosPorDia['data'].tolist(),
    'obitos_confirmados': pf_obitosPorDia['obitosNovos'].tolist()
})

# Criando uma nova coluna no DataFrame referente a média movel.
pf_obitosPorDia['media_movel'] = pf_obitosPorDia.obitos_confirmados.rolling(7).mean()

# Setando a coluna 'data' como índice do DataFrame 
pf_obitosPorDia.set_index('data', inplace=True)


# Processando os dados sobre o Numero de Obitos por Dia (DadosFederais)

In [23]:
# Criando um data com a coluna 'dataNotificacao' e a coluna 'saidaConfirmadaObitos'
df_obitosPorDiaeCNES = dados_federais_campina[['dataNotificacao', 'saidaConfirmadaObitos']]

In [24]:
df_obitosPorDia = {}

for data in df_obitosPorDiaeCNES['dataNotificacao']:
    if data not in df_obitosPorDia:
        ocorrenciasDatas = df_obitosPorDiaeCNES.loc[df_obitosPorDiaeCNES['dataNotificacao'] == data]
        df_obitosPorDia[data] = ocorrenciasDatas['saidaConfirmadaObitos'].sum()

In [25]:
# Criando um DataFrame com os dados para o altair
df_obitosPorDia = pd.DataFrame({
    'data': df_obitosPorDia.keys(),
    'obitos_confirmados': df_obitosPorDia.values()
})

# Criando uma nova coluna no DataFrame referente a média movel.
df_obitosPorDia['media_movel'] = df_obitosPorDia.obitos_confirmados.rolling(7).mean()

# Setando a coluna 'data' como índice do DataFrame 
df_obitosPorDia.set_index('data', inplace=True)

# Processando os sobre o Numero de Obitos por Dia (Dados Municipais)

In [26]:
# Criando um data com a coluna 'data' e a coluna 'obitosNovos'
dm_obitosPorDia = dados_municipais_campina['obitosPorDia']

In [27]:
# Criando um DataFrame com os dados para o altair
dm_obitosPorDia = pd.DataFrame({
    'data': dm_obitosPorDia.index.tolist(),
    'obitos_confirmados': dm_obitosPorDia.values.tolist()
})

# Criando uma nova coluna no DataFrame referente a média movel.
dm_obitosPorDia['media_movel'] = dm_obitosPorDia.obitos_confirmados.rolling(7).mean()

# Setando a coluna 'data' como índice do DataFrame 
dm_obitosPorDia.set_index('data', inplace=True)


In [33]:
def exclui_nao_contidos(df_first, df_second, ambos):
    for i in df_first.index:
        if i not in df_second.index:
            df_first = df_first.drop(i)
    
    if (ambos):
        df_second, df_first = exclui_nao_contidos(df_second, df_first, False)
    return df_first, df_second

In [34]:
pf_obitosPorDia, df_obitosPorDia = exclui_nao_contidos(pf_obitosPorDia,df_obitosPorDia,True)

In [35]:
pf_obitosPorDia

,obitos_confirmados,media_movel
data,,
2021-06-01,4,NaN
2021-06-02,5,NaN
2021-06-03,3,NaN
2021-06-04,3,NaN
2021-06-05,7,NaN
...,...,...
2021-08-27,1,1.000000
2021-08-28,1,0.571429
2021-08-29,0,0.571429


In [36]:
df_obitosPorDia

,obitos_confirmados,media_movel
data,,
2021-06-01,2.0,NaN
2021-06-02,6.0,NaN
2021-06-03,2.0,NaN
2021-06-04,1.0,NaN
2021-06-05,5.0,NaN
...,...,...
2021-08-27,2.0,2.714286
2021-08-28,2.0,2.428571
2021-08-29,0.0,2.428571


In [37]:
pf_obitosPorDia = exclui_nao_contidos(pf_obitosPorDia, dm_obitosPorDia)
df_obitosPorDia = exclui_nao_contidos(df_obitosPorDia, dm_obitosPorDia)

TypeError: exclui_nao_contidos() missing 1 required positional argument: 'ambos'

# Comparando os dados Federais e Municipais

In [ ]:
obitosPorDia = pd.DataFrame({
  'data': pf_obitosPorDia.index.tolist(),
  'Painel Federal': pf_obitosPorDia['obitos_confirmados'].tolist(),
  'Federal': df_obitosPorDia['obitos_confirmados'].tolist(),
  'Municipal': dm_obitosPorDia['obitos_confirmados'].tolist()
}).melt(id_vars=['data'], value_vars=['Painel Federal', 'Federal', 'Municipal'],
        var_name='fonte_dados', value_name='aplic_por_dia')

In [ ]:
alt.Chart(obitosPorDia).mark_line().encode(
    alt.X('data:T', title='Data'),
    alt.Y('aplic_por_dia:Q', title='Nº de Obitos'),
    alt.Color('fonte_dados:N', title='Base dos Dados')
).properties(
    width=700,
    title='Quantidade de Obitos Confirmados por Dia'
)

alt.Chart(...)

In [ ]:
obitosPorDiaMedMovel = pd.DataFrame({
  'data': pf_obitosPorDia.index.tolist(),
  'Painel Federal': pf_obitosPorDia['media_movel'].tolist(),
  'Federal': df_obitosPorDia['media_movel'].tolist(),
  'Municipal': dm_obitosPorDia['media_movel'].tolist()
}).melt(id_vars=['data'], value_vars=['Painel Federal', 'Federal', 'Municipal'],
        var_name='fonte_dados', value_name='aplic_por_dia')

In [ ]:
alt.Chart(obitosPorDiaMedMovel).mark_line(point=True).encode(
    alt.X('data:T', title='Data'),
    alt.Y('aplic_por_dia:Q', title='Nº de Obitos'),
    alt.Color('fonte_dados:N', title='Base dos Dados')
).properties(
    width=750,
    title='Média Movel de Obitos Confirmados por Dia (Comparação)'
)

alt.Chart(...)

In [ ]:
pf_obitosTotais = pf_obitosPorDia['obitos_confirmados'].sum()
df_obitosTotais = df_obitosPorDia['obitos_confirmados'].sum()
dm_obitosTotais = dm_obitosPorDia['obitos_confirmados'].sum()

In [ ]:
totalObitos = pd.DataFrame({
    'base': ['Painel Federal', 'Federal', 'Municipal'],
    'total': [pf_obitosTotais, df_obitosTotais, dm_obitosTotais]
})

In [ ]:
alt.Chart(totalObitos).mark_bar(size=30).encode(
    alt.X('base:N', title='Base dos Dados'),
    alt.Y('total:Q', title='Nº Total de Obitos Confirmados'),
    alt.Color('base:N', title='Base de Dados')
).properties(
    width=150,
    title='Total de Obitos Confirmados (Comparação)'
)

alt.Chart(...)